# Notebook Objective and Setup

BGG03 is the scrubbing and cleaning of the various data obtained in notebooks BGG01 and BGG02. The following datasets are cleaned, constructed, or otherwise prepared for EDA and modeling.

    * Games
   * Mechanics
    * Subcategories
    * Designers
    * Artists
    * Publishers
    * Awards
    * Ratings Distribution
    * Comments
    * Ratings Matrix

## Package Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc
import json
from statistics import mean

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# NLP tools
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

## Notebook Functions

In [2]:
def integer_reduce(data, columns, fill_value=0):
    '''
    Reduces an integer type to its smallest memory size type
    
    Inputs:
    data: dataframe to reduce
    columns: columns to reduce
    fill_value: fill value to use if none
    
    Returns: 
    data: dataframe with memory reduced data types
    '''
    for column in columns:
        print(column)
        data[column].fillna(fill_value, inplace=True)
        if (data[column].max() <= 127) & (data[column].min() >= -128):
            data[column] = data[column].astype('int8')
        elif (data[column].max() <= 32767) & (data[column].min() >= -32768):
            data[column] = data[column].astype('int16')
        elif (data[column].max() <= 2147483647) & (data[column].min() >= -2147483648):
            data[column] = data[column].astype('int32')
        
    return data

In [3]:
def text_block_processor(text):
    '''Takes a block of text. Divides block into sentences with words lemmatized.
    Sends each sentence to word processor. Concatenates all words into one string
    Otherwise returns string of cleaned and processed words from text block
    
    ARGUMENTS:
    block of text
    '''
    
    text = str(text)
    line = re.sub(r'[^a-zA-Z\s]', '', text).lower() # removes all special characters and numbers, and makes lower case
    line2 = re.sub(r'\s{2}', '', line).lower() # removes extra blocks of 2 spaces
    tokens = nlp(line)
    words = []
    for token in tokens:
        if token.is_stop == False:
            token_preprocessed = token.lemma_
            if token_preprocessed != '': # only continues if returned word is not empty
                words.append(token_preprocessed) # appends word to list of words
    line = ' '.join(words)
    
    return line


In [4]:
def fix_numbers(x):
    '''
    Checks for numbers or strings
    If a string, strips off the "k" and multiply by 10000
    Sends back cleaned int
    '''
    
    if type(x) is int:
        return int(x)
    
    if str.endswith(x, 'k'):
        x = str(x).strip('k')
        new_num = int(float(x)*1000)
        return int(new_num)
    
    else: 
        return int(x)

In [5]:
def clean_ratings(id_num, game_ids):
    '''
    Loads and cleans a raw user ratings file
    Drops game ids not present in games file
    Drops users with fewer than 10 ratings
    
    Inputs:
    id_num: the appendation of the file to find the path
    game_ids: list of game ids in the games file
    
    Outputs:
    Cleaned user ratings file
    '''
    
    print('\nCleaning Frame #'+str(id_num))
    
    #load in raw users file according to id_num inputted
    path = 'userid/user_ratings'+str(id_num)+'.pkl'
    users = pd.read_pickle(path)
    
    # convert all datatypes to float
    float_converted = users.astype('float')
    
    # delete and clean up raw users file
    del users
    gc.collect()
    
    # create intersection between user file and game list ids
    float_converted.columns = float_converted.columns.astype('int32')
    cleaned = float_converted[float_converted.columns.intersection(game_ids)]
    
    # delete and clean up
    del float_converted
    gc.collect()
    
    # make a list of users with fewer than 5 user ratings
    sums = cleaned.count(axis=1)<5
    # get indices for the rows with fewer than 5 ratings
    drop_these = sums.loc[sums==True].index
    # drop the users with fewer than 5 ratings
    cleaned.drop(drop_these, axis=0, inplace=True)
    
    
    # print memory usage
    print(cleaned.info())
    
    # return cleaned file
    return cleaned

In [6]:
def create_ratings_file(start_file, end_file, game_ids):
    '''
    Puts together dataframes from a range of files
    Each file calls the clean_ratings function
    Then all files in range are concatenated
    
    Inputs:
    start_file: start of file name appendation
    end_file: end file name appendation
    game_ids_list: list of game ids in the games file
    
    Outputs:
    Cleaned and concatenated master file
    
    '''
    
    # make an empty dataframe
    master_file = pd.DataFrame()
    
    # for each number in the range from start to end:
    for id_num in np.arange(start_file,end_file+1,1):
        print(id_num)
        # clean the file calling clean_ratings
        cleaned_item = clean_ratings(id_num, game_ids)
        # append the file to the dataframe
        master_file = pd.concat([master_file, cleaned_item], axis=0)
    
    master_file.drop_duplicates(keep='first', inplace=True)
    
    # clean up
    del cleaned_item
    gc.collect()
    
    return master_file

In [7]:
def process_dataframe_ratings(x, user_ratings, raw_ratings):
    
    try:
        user_ratings[x['Username']][x['BGGId']] = float(x['Rating']) 
    
    except:
        user_ratings[x['Username']] = {}
        user_ratings[x['Username']][x['BGGId']] = float(x['Rating']) 
    
    raw_ratings[x['BGGId']].append(x['Rating'])

# Games Files

In [8]:
games = pd.read_pickle('data_dirty/games.pkl')
games.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,ImagePath,Rank:boardgame,Rank:strategygames,Family,Theme,Kickstarted,Rank:thematic,Mechanism,Category,Setting,Rank:wargames,Rank:familygames,Rank:cgs,Rank:abstracts,Rank:partygames,Rank:childrensgames,Rank:rpgitem,Rank:boardgameaccessory,Rank:videogame,Rank:amiga,Rank:commodore64,Rank:arcade,Rank:atarist
0,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,2018,3.8839,8.59990,8.41718,1.41682,2,4,13.316129,None,3,"[2, 3, 4]",61375,1728,17573,2127,120,60,120,14,44160,6,0,1,1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,1.0,1.0,Brass,Canals,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,2015,2.8323,8.53017,8.38327,1.59853,2,4,11.347150,None,4,"[2, 3, 4]",82185,815,13558,1437,60,60,60,13,52895,11,0,2,1,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2.0,2.0,Pandemic,Scienc,NaN,1.0,Campaign G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,2017,3.9072,8.59724,8.36260,1.73911,1,4,12.830721,None,3,"[1, 2, 3, 4]",95933,1194,20149,2531,120,60,120,14,61423,6,17,1,0,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,3.0,4.0,Gloomhaven,NaN,1.0,2.0,Campaign G,Dungeon Crawl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,342942,Ark Nova,"In Ark Nova, you will plan and design a modern...",2021,3.7493,8.53828,8.33379,1.37673,1,4,12.714286,None,2,"[1, 2, 3]",60338,1040,12879,2154,150,90,150,14,40992,8,2,0,0,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,4.0,3.0,Ark Nov,Ecology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,Twilight Imperium (Fourth Edition) is a game o...,2017,4.3173,8.60539,8.24226,1.61735,3,6,14.303030,None,6,"[4, 5, 6]",28368,1034,11264,1122,480,240,480,14,23152,11,4,1,1,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,5.0,5.0,Twilight Imperiu,NaN,NaN,3.0,4X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Get info, make note of datatypes and memory usage
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    40756 non-null  int64  
 1   Name                     40756 non-null  object 
 2   Description              40756 non-null  object 
 3   YearPublished            40756 non-null  int64  
 4   GameWeight               40756 non-null  float64
 5   AvgRating                40756 non-null  float64
 6   BayesAvgRating           40756 non-null  float64
 7   StdDev                   40756 non-null  float64
 8   MinPlayers               40756 non-null  int64  
 9   MaxPlayers               40756 non-null  int64  
 10  ComAgeRec                24496 non-null  float64
 11  LanguageEase             0 non-null      object 
 12  BestPlayers              2580 non-null   object 
 13  GoodPlayers              40756 non-null  object 
 14  NumOwned              

In [10]:
# drop duplicate entires
games = games.drop_duplicates(subset='BGGId', keep='first')

In [11]:
drop_columns = ['NumAwards', 'NumFans', 'NumPageViews', 'RulesPosts', 'TotalPosts', 'Category', 'IsExpansion', 'Rank:rpgitem', 'Rank:boardgameaccessory', 'Rank:videogame', 'Rank:amiga', 'Rank:commodore64', 'Rank:arcade', 'Rank:atarist', 'Setting', 'Mechanism']

# drop non-boardgame related information
for column in drop_columns:
    if column in games.columns:
        games = games.drop(column, axis=1)
        
games.shape # check shape of file

(40756, 40)

In [12]:
# what are the columns?
games.columns

Index(['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumAlternates',
       'NumExpansions', 'NumImplementations', 'IsReimplementation',
       'ImagePath', 'Rank:boardgame', 'Rank:strategygames', 'Family', 'Theme',
       'Kickstarted', 'Rank:thematic', 'Rank:wargames', 'Rank:familygames',
       'Rank:cgs', 'Rank:abstracts', 'Rank:partygames', 'Rank:childrensgames'],
      dtype='object')

In [13]:
games['BestPlayers'] = games['BestPlayers'].fillna(0) # Fill NaN on BestPlayers
games['BestPlayers'].unique() # What are the unique BestPlayers entries?

array(['3', '4', '2', '6', '1', '5', '7', '12', '8', 0, '3+', '14', '15',
       '9', '13', '4+', '0+'], dtype=object)

In [14]:
# Clean up Best Players so all are integers
games.loc[(games['BestPlayers']=='3+'), 'BestPlayers'] = 3
games.loc[(games['BestPlayers']=='0+'), 'BestPlayers'] = 4

# change dtype on BestPlayers to int8
games['BestPlayers'] = games['BestPlayers'].astype('int8')

ValueError: invalid literal for int() with base 10: '4+'

In [ ]:
# Add Categories with binary flags
games.loc[games['Rank:thematic'].notna(), 'Cat:Thematic'] = 1
games.loc[games['Rank:strategygames'].notna(), 'Cat:Strategy'] = 1
games.loc[games['Rank:wargames'].notna(), 'Cat:War'] = 1
games.loc[games['Rank:familygames'].notna(), 'Cat:Family'] = 1
games.loc[games['Rank:cgs'].notna(), 'Cat:CGS'] = 1
games.loc[games['Rank:abstracts'].notna(), 'Cat:Abstract'] = 1
games.loc[games['Rank:partygames'].notna(), 'Cat:Party'] = 1
games.loc[games['Rank:childrensgames'].notna(), 'Cat:Childrens'] = 1

In [ ]:
# prepare different column sets for memory integer reduction

# integer reduction with fill_values of 0
int_columns=['BGGId', 'YearPublished', 'MinPlayers', 'MaxPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime','MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Kickstarted', 'Cat:Thematic', 'Cat:Strategy', 
        'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']

# integer reduction with fill_values of 21926 (lower is better on these)
ranks = ['Rank:boardgame', 'Rank:thematic', 'Rank:strategygames',
       'Rank:wargames', 'Rank:familygames', 'Rank:cgs', 'Rank:abstracts',
       'Rank:partygames', 'Rank:childrensgames']

# call integer_reduce on the sets
games = integer_reduce(games, int_columns, fill_value=0)

games = integer_reduce(games, ranks, fill_value=21926)

games.info() # recheck data types and memory usage

In [ ]:
# any games with no name listed?
games.loc[games['Name']==0]

In [15]:
# if so, drop them and reset index
#empty_games = list(games.loc[games['Name']==0].index)
#games.drop(games.index[empty_games], inplace=True)
#games.reset_index(inplace=True, drop=True)

In [16]:
# Drop all games that are not yet released, then reset index
#not_released = list(games.loc[games['YearPublished']>2021].index)
#games.drop(games.index[not_released], inplace=True)
#games.reset_index(inplace=True, drop=True)

In [17]:
themes = pd.DataFrame(games['Theme'])
games.drop('Theme', axis=1, inplace=True)

In [18]:
# process the Description column text
games['Description'] = games['Description'].apply(lambda x: text_block_processor(x))

In [19]:
games.reset_index(inplace=True, drop=True)

In [20]:
# save to file
games.to_csv('data_kaggle/games.csv', index=False)

In [21]:
# All games with over 12 players are set at 13 players.
games.loc[games['MaxPlayers']>12, 'MaxPlayers'] = 13

In [22]:
# Games with min players of 0, we will set their min players = 2
games.loc[games['MinPlayers']<1, 'MaxPlayers'] = 2

In [24]:
# save to file
games.to_pickle('data_cleaned/games.pkl')

In [25]:
games.shape

(40756, 39)

## Game ID Lookup

In [26]:
# dictionary of game IDs-Names

# Load games
games = pd.read_pickle('data_cleaned/games.pkl')

# lists of game ids and game names
game_ids = list(games['BGGId'])
game_names = list(games['Name'])

# make lookup dictionary
game_id_lookup = {}

# store ids and names in lookup dictionary
for key, item in zip(game_ids, game_names):
    game_id_lookup[key] = item

    
#del games
#gc.collect()

game_id_lookup

{224517: 'Brass: Birmingham',
 161936: 'Pandemic Legacy: Season 1',
 174430: 'Gloomhaven',
 342942: 'Ark Nova',
 233078: 'Twilight Imperium: Fourth Edition',
 316554: 'Dune: Imperium',
 167791: 'Terraforming Mars',
 115746: 'War of the Ring: Second Edition',
 291457: 'Gloomhaven: Jaws of the Lion',
 187645: 'Star Wars: Rebellion',
 162886: 'Spirit Island',
 220308: 'Gaia Project',
 12333: 'Twilight Struggle',
 182028: 'Through the Ages: A New Story of Civilization',
 193738: 'Great Western Trail',
 84876: 'The Castles of Burgundy',
 169786: 'Scythe',
 246900: 'Eclipse: Second Dawn for the Galaxy',
 173346: '7 Wonders Duel',
 28720: 'Brass: Lancashire',
 167355: 'Nemesis',
 266507: 'Clank! Legacy: Acquisitions Incorporated',
 124361: 'Concordia',
 177736: 'A Feast for Odin',
 266192: 'Wingspan',
 120677: 'Terra Mystica',
 205637: 'Arkham Horror: The Card Game',
 312484: 'Lost Ruins of Arnak',
 341169: 'Great Western Trail: Second Edition',
 237182: 'Root',
 164928: 'Orléans',
 295770: '

In [27]:
# save dictionary
with open('data_cleaned/game_id_lookup.json', 'w') as convert_file:
    convert_file.write(json.dumps(game_id_lookup))

## Mechanics and Subcategories

### Clean Mechanics

In [28]:
# Load mechanics and check memory usage
mechanics = pd.read_pickle('data_dirty/mechanics.pkl')
mechanics

,BGGId,Hand Management,Income,Loans,Market,Network and Route Building,Tags,Tech Trees / Tech Tracks,Turn Order: Stat-Based,Variable Set-up,Action Points,Cooperative Game,Legacy Game,Point to Point Movement,Scenario / Mission / Campaign Game,Set Collection,Trading,Variable Player Powers,Legacy,Action Queue,Action Retrieval,Campaign / Battle Card Driven,Card Play Conflict Resolution,Communication Limits,Critical Hits and Failures,Deck Construction,Grid Movement,Hexagon Grid,Line of Sight,Modular Board,Multi-Use Cards,Once-Per-Game Abilities,Role Playing,Simultaneous Action Selection,Solo / Solitaire Game,End Game Bonuses,Increase Value of Unchosen Resources,Open Drafting,Tile Placement,Action Drafting,Area-Impulse,Dice Rolling,Follow,King of the Hill,Race,Variable Phase Order,Voting,"Deck, Bag, and Pool Building",Delayed Purchase,Force Commitment,Take That,Turn Order: Progressive,Worker Placement,Closed Drafting,Contracts,Enclosure,Victory Points as a Resource,TableauBuilding,Area Majority / Influence,Area Movement,Events,Movement Points,Team-Based Game,Die Icon Resolution,Hidden Movement,"Worker Placement, Different Worker Types",Automatic Resource Growth,Turn Order: Pass Order,Action/Event,Advantage Token,Simulation,Sudden Death Ending,Tug of War,Auction: Dutch,Auction/Bidding,Ownership,Track Movement,Grid Coverage,Pattern Building,Zone of Control,Alliances,Passed Action Token,Layering,Hidden Roles,Interrupts,Player Elimination,Semi-Cooperative Game,Traitor Game,Map Addition,Narrative Choice / Paragraph,Pick-up and Deliver,Push Your Luck,Hidden Victory Points,Investment,Chaining,Map Deformation,Stat Check Resolution,Resource to Move,Square Grid,Turn Order: Role Order,Turn Order: Claim Action,Trick-taking,Turn Order: Auction,Negotiation,Score-and-Reset Game,Catch the Leader,Programmed Movement,Flicking,Map Reduction,Memory,Moving Multiple Units,Storytelling,Stock Holding,Bias,Auction: Turn Order Until Pass,Commodity Speculation,Slide/Push,Secret Unit Deployment,Constrained Bidding,Mancala,Chit-Pull System,Pattern Recognition,Highest-Lowest Scoring,Rondel,Worker Placement with Dice Workers,Neighbor Scope,Deduction,Paper-and-Pencil,Turn Order: Time Track,Selection Order Bid,Targeted Clues,Auction: Fixed Placement,Auction: Multiple Lot,Connections,Ladder Climbing,Auction: Sealed Bid,Betting and Bluffing,Order Counters,Auction: Once Around,Closed Economy Auction,Bingo,Line Drawing,Three Dimensional Movement,Movement Template,Re-rolling and Locking,Roll / Spin and Move,Resource Queue,Minimap Resolution,Multiple Maps,Command Cards,Bribery,Predictive Bid,Prisoner's Dilemma,"I Cut, You Choose",Finale Ending,Roles with Asymmetric Information,Rock-Paper-Scissors,Random Production,Real-Time,Auction: English,Kill Steal,Relative Movement,Cube Tower,Move Through Deck,Melding and Splaying,Acting,Singing,Pattern Movement,Pieces as Map,Drawing,Static Capture,Elapsed Real Time Ending,Ordering,Auction Compensation,Turn Order: Random,Lose a Turn,Bids As Wagers,Ratio / Combat Results Table,Stacking and Balancing,Spelling,Different Dice Movement,Physical Removal,Measurement Movement,Single Loser Game,Player Judge,Auction: Dutch Priority,Action Timer,Hot Potato,Induction,Speed Matching,Matching,Questions and Answers,Crayon Rail System,Impulse Movement,Auction: Dexterity
0,224517,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [29]:
mechanics.drop_duplicates(subset='BGGId', keep='first', inplace=True)  # drop duplicates
mechanics.fillna(0, inplace=True) # fill nan

mechanics.reset_index(inplace=True, drop=True) # reset index
mechanics.info() # check memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 195 entries, BGGId to Auction: Dexterity
dtypes: float64(194), int64(1)
memory usage: 60.6 MB


In [48]:
list(mechanics.columns)

['BGGId',
 'Hand Management',
 'Income',
 'Loans',
 'Market',
 'Network and Route Building',
 'Tags',
 'Tech Trees / Tech Tracks',
 'Turn Order: Stat-Based',
 'Variable Set-up',
 'Action Points',
 'Cooperative Game',
 'Legacy Game',
 'Point to Point Movement',
 'Scenario / Mission / Campaign Game',
 'Set Collection',
 'Trading',
 'Variable Player Powers',
 'Legacy',
 'Action Queue',
 'Action Retrieval',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Communication Limits',
 'Critical Hits and Failures',
 'Deck Construction',
 'Grid Movement',
 'Hexagon Grid',
 'Line of Sight',
 'Modular Board',
 'Multi-Use Cards',
 'Once-Per-Game Abilities',
 'Role Playing',
 'Simultaneous Action Selection',
 'Solo / Solitaire Game',
 'End Game Bonuses',
 'Increase Value of Unchosen Resources',
 'Open Drafting',
 'Tile Placement',
 'Action Drafting',
 'Area-Impulse',
 'Dice Rolling',
 'Follow',
 'King of the Hill',
 'Race',
 'Variable Phase Order',
 'Voting',
 'Deck, Bag, and Pool

In [54]:
auction_mechanics = mechanics[[x for x in mechanics.columns if 'legacy' in x.lower()]].columns.to_list()
auction_mechanics

['Legacy Game', 'Legacy']

In [56]:
# Clean up mechanics
# Here we are using our domain knowledge to compact several different catogories into one

auction_list = mechanics[[x for x in mechanics.columns if 'auction' in x.lower()]].columns.to_list()

turn_order_list = mechanics[[x for x in mechanics.columns if 'turn order' in x.lower()]].columns.to_list()

drafting = mechanics[[x for x in mechanics.columns if 'drafting' in x.lower()]].columns.to_list()

legacy = ['Legacy']

worker_placement = mechanics[[x for x in mechanics.columns if 'worker' in x.lower()]].columns.to_list()


# compacting categories here:

for item in worker_placement:
    mechanics.loc[mechanics[item]==1, 'Worker Placement'] = 1
    mechanics.drop([item], axis=1, inplace=True)

for item in auction_list:
    mechanics.loc[mechanics[item]==1, 'Auction/Bidding'] = 1
    mechanics.drop([item], axis=1, inplace=True)

for item in drafting:
    mechanics.loc[mechanics[item]==1, 'Drafting'] = 1
    mechanics.drop([item], axis=1, inplace=True)

mechanics.loc[mechanics['Legacy']==1, 'Legacy Game'] = 1

mechanics.drop(turn_order_list, axis=1, inplace=True)
mechanics.drop(['Legacy'], axis=1, inplace=True)

In [57]:
columns = mechanics.columns

# call integer_reduce on the sets
mechanics = integer_reduce(mechanics, columns, fill_value=0)

BGGId
Hand Management
Income
Loans
Market
Network and Route Building
Tags
Tech Trees / Tech Tracks
Variable Set-up
Action Points
Cooperative Game
Legacy Game
Point to Point Movement
Scenario / Mission / Campaign Game
Set Collection
Trading
Variable Player Powers
Action Queue
Action Retrieval
Campaign / Battle Card Driven
Card Play Conflict Resolution
Communication Limits
Critical Hits and Failures
Deck Construction
Grid Movement
Hexagon Grid
Line of Sight
Modular Board
Multi-Use Cards
Once-Per-Game Abilities
Role Playing
Simultaneous Action Selection
Solo / Solitaire Game
End Game Bonuses
Increase Value of Unchosen Resources
Tile Placement
Area-Impulse
Dice Rolling
Follow
King of the Hill
Race
Variable Phase Order
Voting
Deck, Bag, and Pool Building
Delayed Purchase
Force Commitment
Take That
Contracts
Enclosure
Victory Points as a Resource
TableauBuilding
Area Majority / Influence
Area Movement
Events
Movement Points
Team-Based Game
Die Icon Resolution
Hidden Movement
Automatic Resour

In [59]:
mechanics.head()

,BGGId,Hand Management,Income,Loans,Market,Network and Route Building,Tags,Tech Trees / Tech Tracks,Variable Set-up,Action Points,Cooperative Game,Legacy Game,Point to Point Movement,Scenario / Mission / Campaign Game,Set Collection,Trading,Variable Player Powers,Action Queue,Action Retrieval,Campaign / Battle Card Driven,Card Play Conflict Resolution,Communication Limits,Critical Hits and Failures,Deck Construction,Grid Movement,Hexagon Grid,Line of Sight,Modular Board,Multi-Use Cards,Once-Per-Game Abilities,Role Playing,Simultaneous Action Selection,Solo / Solitaire Game,End Game Bonuses,Increase Value of Unchosen Resources,Tile Placement,Area-Impulse,Dice Rolling,Follow,King of the Hill,Race,Variable Phase Order,Voting,"Deck, Bag, and Pool Building",Delayed Purchase,Force Commitment,Take That,Contracts,Enclosure,Victory Points as a Resource,TableauBuilding,Area Majority / Influence,Area Movement,Events,Movement Points,Team-Based Game,Die Icon Resolution,Hidden Movement,Automatic Resource Growth,Action/Event,Advantage Token,Simulation,Sudden Death Ending,Tug of War,Ownership,Track Movement,Grid Coverage,Pattern Building,Zone of Control,Alliances,Passed Action Token,Layering,Hidden Roles,Interrupts,Player Elimination,Semi-Cooperative Game,Traitor Game,Map Addition,Narrative Choice / Paragraph,Pick-up and Deliver,Push Your Luck,Hidden Victory Points,Investment,Chaining,Map Deformation,Stat Check Resolution,Resource to Move,Square Grid,Trick-taking,Negotiation,Score-and-Reset Game,Catch the Leader,Programmed Movement,Map Reduction,Memory,Moving Multiple Units,Storytelling,Stock Holding,Bias,Commodity Speculation,Slide/Push,Secret Unit Deployment,Constrained Bidding,Mancala,Chit-Pull System,Pattern Recognition,Highest-Lowest Scoring,Rondel,Neighbor Scope,Deduction,Paper-and-Pencil,Selection Order Bid,Targeted Clues,Connections,Ladder Climbing,Betting and Bluffing,Order Counters,Bingo,Line Drawing,Three Dimensional Movement,Movement Template,Re-rolling and Locking,Roll / Spin and Move,Resource Queue,Minimap Resolution,Multiple Maps,Command Cards,Bribery,Predictive Bid,Prisoner's Dilemma,"I Cut, You Choose",Finale Ending,Roles with Asymmetric Information,Random Production,Real-Time,Kill Steal,Relative Movement,Cube Tower,Move Through Deck,Melding and Splaying,Pattern Movement,Pieces as Map,Drawing,Static Capture,Elapsed Real Time Ending,Ordering,Lose a Turn,Bids As Wagers,Ratio / Combat Results Table,Spelling,Different Dice Movement,Physical Removal,Measurement Movement,Single Loser Game,Player Judge,Action Timer,Induction,Speed Matching,Matching,Questions and Answers,Crayon Rail System,Impulse Movement,Dexterity,Physical
0,224517,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,161936,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,174430,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,342942,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Clean Subcategories

In [60]:
themes_expanded = pd.get_dummies(themes)
theme_sort = pd.DataFrame(themes_expanded.sum().sort_values(ascending=False))
themes_over_1 = list(theme_sort.loc[theme_sort[0]>1].index)
themes_attach = themes_expanded[themes_over_1]
themes_attach.head()

,Theme_Food / Cooking,Theme_Anime / Manga,Theme_Superheroes,Theme_Anthropomorphic Animals,Theme_Mystery / Cri,Theme_Alternate History,Theme_Cthulhu Mythos,Theme_Vikings,Theme_Colonial,Theme_Post-Apocalyptic,Theme_Robots,Theme_Time Travel,Theme_Witches,Theme_Archaeology / Paleontology,Theme_Sieg,Theme_Mad Science / Mad Scientist,Theme_Art,Theme_Natur,Theme_Pirates,Theme_Mining,Theme_Gardening,Theme_Retro,Theme_Steampunk,Theme_Burglary and Heists,Theme_Native Americans / First Peoples,Theme_Deserts,Theme_Flowers,Theme_Fishing / Angling,Theme_Cyberpunk,Theme_Submarines,Theme_Construction,Theme_Ninjas,Theme_Circus,Theme_Amusement Parks / Theme Parks,Theme_Tropical,Theme_King Arthur / The Knights of the Round Table / Camelot,Theme_Spooky Old Houses,Theme_Biology,Theme_Love / Romanc,Theme_Gladiators,Theme_Kaiju,Theme_Samurai,Theme_Weather,Theme_Villainy,Theme_Books / Libraries,Theme_Fantasy Sports,Theme_Safaris,Theme_Boss Battl,Theme_City,Theme_Arcade Video Games,Theme_Airships / Blimps / Dirigibles / Zeppelins,Theme_Mech Warfar,Theme_Alchemy,Theme_Survival,Theme_Movie Industry,Theme_Oil / Gas / Petroleu,Theme_Scienc,Theme_Dreams / Nightmares,Theme_Boardgaming,Theme_Polic,Theme_Excrements / Feces / Poop / Urin,Theme_Art style – Art Deco,Theme_Beaches,Theme_Trees and Forests,Theme_Psychology,Theme_Climate Chang,Theme_Animal Battles,Theme_Music Making & Makers,Theme_Nuclear option,Theme_Ecology,Theme_Tropical Islands,Theme_Evolution,Theme_Under the Sea,Theme_Video Game Theme: Nintendo,Theme_Mail / Stamps / The Post Offic,Theme_Firefighting,Theme_Romance of the Three Kingdoms,Theme_Volcanoes,Theme_Fictional Games,Theme_Hell,Theme_Attorneys / Courts,Theme_Jail / Prison (Modern),Theme_Treasure Diving,Theme_Hackers,Theme_Sci-Fi Sports,Theme_Aztecs,Theme_Astronomy,Theme_Helicopters,Theme_Chibis,Theme_Mayans,Theme_Trucks,Theme_Chemistry,Theme_Computer / Information Technology Industry,Theme_Automotive Industry,Theme_Sewing / Knitting / Cloth-Making,Theme_US National Parks,Theme_Trash / Garbag,Theme_Movies,Theme_Dolls,Theme_Mushrooms,Theme_Motorcycles,Theme_Hot Air Balloons,Theme_Plague Epidemic,Theme_Art style – Art Nouveau,"Theme_Pub, Bars, Bistros...",Theme_Laundry & Socks,Theme_Smuggling,Theme_Chivalry / Jousting / Tournaments (Medieval Europe),Theme_Canals,Theme_Silk Road,Theme_TV Detectives,Theme_Stock Market,"Theme_Zoos, Aquaria, Safari Parks",Theme_Environmental Protection / Degradation / Pollution,Theme_Journalis,Theme_My Best Lif,Theme_Battle Royal,Theme_Camping,Theme_Memes,Theme_African Americans,Theme_Traffic / Driving,Theme_Physics,Theme_Jewelry,Theme_Psychic Powers,Theme_Druids,Theme_UFOs,Theme_Cannibals / Cannibalis,Theme_Hanseatic Leagu,Theme_Cacti and Succulents,Theme_Floating islands in the sky,Theme_Cereal Games,Theme_Cemeteries / Graveyards,Theme_Endangered / Extinct species,Theme_Photography,Theme_Business/Management,Theme_Hiking,Theme_Infiltration,Theme_Video Game Theme: Tetris,Theme_Video Game Theme: Pokémon,Theme_Pulp,Theme_Video Game Theme: Super Mario Bros.,Theme_FIFA World Cup,Theme_Teaching Programming,Theme_Fine Art and Art Museums,Theme_Television (TV) Industry,Theme_Cryptids,Theme_Disney Theme Parks,Theme_Templ,Theme_Perfu,Theme_Māori,Theme_Fireworks,Theme_Rubik's Cub,Theme_Astrology,Theme_Fashion,Theme_School / College / University,Theme_Tiki Cultur,Theme_Queer / LGBTAIQ,Theme_Video Game Theme: Resident Evil,Theme_Care Bears,Theme_Birthdays,Theme_Spanish Political Games,Theme_Latin American Political Games,Theme_Orient Express,Theme_Electricity Generation,Theme_Painting / Paintings,Theme_Earthquakes,Theme_Knights Templar,Theme_Video Game Theme: SEGA,Theme_Video Game Theme: Minecraft,Theme_Bacteria,Theme_Video Game Theme: Carmen Sandiego,Theme_Rivers,Theme_Inuit Peoples,Theme_Dieselpunk,Theme_Video Game Theme: The Elder Scrolls,Theme_Geisha,Theme_Video Game Theme: Angry Birds,Theme_Video Game Theme: Sonic the Hedgehog,Theme_Geocaching,Theme_Flags identification,Theme_Clowns,Theme_Cruise ships,Theme_Metal Smithi

In [61]:
columns = themes_attach.columns

# call integer_reduce on the sets
themes_attach = integer_reduce(themes_attach, columns, fill_value=0)

Theme_Food / Cooking
Theme_Anime / Manga
Theme_Superheroes
Theme_Anthropomorphic Animals
Theme_Mystery / Cri
Theme_Alternate History
Theme_Cthulhu Mythos
Theme_Vikings
Theme_Colonial
Theme_Post-Apocalyptic
Theme_Robots
Theme_Time Travel
Theme_Witches
Theme_Archaeology / Paleontology
Theme_Sieg
Theme_Mad Science / Mad Scientist
Theme_Art
Theme_Natur
Theme_Pirates
Theme_Mining
Theme_Gardening
Theme_Retro
Theme_Steampunk
Theme_Burglary and Heists
Theme_Native Americans / First Peoples
Theme_Deserts
Theme_Flowers
Theme_Fishing / Angling
Theme_Cyberpunk
Theme_Submarines
Theme_Construction
Theme_Ninjas
Theme_Circus
Theme_Amusement Parks / Theme Parks
Theme_Tropical
Theme_King Arthur / The Knights of the Round Table / Camelot
Theme_Spooky Old Houses
Theme_Biology
Theme_Love / Romanc
Theme_Gladiators
Theme_Kaiju
Theme_Samurai
Theme_Weather
Theme_Villainy
Theme_Books / Libraries
Theme_Fantasy Sports
Theme_Safaris
Theme_Boss Battl
Theme_City
Theme_Arcade Video Games
Theme_Airships / Blimps / Dir

In [62]:
# load subcategories file and check memory usage

indices = list(games['BGGId'])
subcategories = pd.read_pickle('data_dirty/subcategories.pkl')
subcategories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BGGId                       40756 non-null  int64  
 1   Age of Reason               277 non-null    float64
 2   Economic                    2492 non-null   float64
 3   Industry / Manufacturing    450 non-null    float64
 4   Post-Napoleonic             445 non-null    float64
 5   Trains                      529 non-null    float64
 6   Transportation              639 non-null    float64
 7   Environmental               437 non-null    float64
 8   Medical                     137 non-null    float64
 9   Adventure                   2031 non-null   float64
 10  Exploration                 1462 non-null   float64
 11  Fantasy                     4596 non-null   float64
 12  Fighting                    2694 non-null   float64
 13  Miniatures                  206

In [63]:
subcategories.reset_index(inplace=True, drop=True) # reset index

In [64]:
subcategories.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
subcategories.fillna(0, inplace=True) # fill nan
#subcategories.set_index('BGGId', drop=True, inplace=True) # drop bad games
#subcategories = subcategories.loc[indices] # use only indices in games file

columns = subcategories.columns

# call integer_reduce on the sets
subcategories = integer_reduce(subcategories, columns, fill_value=0)


subcategories.head()

BGGId
Age of Reason
Economic
Industry / Manufacturing
Post-Napoleonic
Trains
Transportation
Environmental
Medical
Adventure
Exploration
Fantasy
Fighting
Miniatures
Animals
Civilization
Negotiation
Political
Science Fiction
Space Exploration
Wargame
Movies / TV / Radio theme
Novel-based
Territory Building
Civil War
Mythology
Modern Warfare
Card Game
American West
Dice
Medieval
Ancient
City Building
Horror
Nautical
Farming
Puzzle
Educational
Collectible Components
Travel
Religious
Deduction
Comic Book / Strip
Spies/Secret Agents
Racing
Sports
Action / Dexterity
Murder/Mystery
Pirates
Video Game Theme
Bluffing
Mature / Adult
Abstract Strategy
Renaissance
Arabian
Prehistoric
Party Game
Word Game
World War II
Aviation / Flight
Number
Pike and Shot
Real-time
Zombies
World War I
Humor
Print & Play
Electronic
Napoleonic
Math
Maze
Mafia
Children's Game
American Indian Wars
American Revolutionary War
Vietnam War
Memory
Expansion for Base-game
American Civil War
Music
Game System
Trivia
Korean Wa

,BGGId,Age of Reason,Economic,Industry / Manufacturing,Post-Napoleonic,Trains,Transportation,Environmental,Medical,Adventure,Exploration,Fantasy,Fighting,Miniatures,Animals,Civilization,Negotiation,Political,Science Fiction,Space Exploration,Wargame,Movies / TV / Radio theme,Novel-based,Territory Building,Civil War,Mythology,Modern Warfare,Card Game,American West,Dice,Medieval,Ancient,City Building,Horror,Nautical,Farming,Puzzle,Educational,Collectible Components,Travel,Religious,Deduction,Comic Book / Strip,Spies/Secret Agents,Racing,Sports,Action / Dexterity,Murder/Mystery,Pirates,Video Game Theme,Bluffing,Mature / Adult,Abstract Strategy,Renaissance,Arabian,Prehistoric,Party Game,Word Game,World War II,Aviation / Flight,Number,Pike and Shot,Real-time,Zombies,World War I,Humor,Print & Play,Electronic,Napoleonic,Math,Maze,Mafia,Children's Game,American Indian Wars,American Revolutionary War,Vietnam War,Memory,Expansion for Base-game,American Civil War,Music,Game System,Trivia,Korean War,Fan Expansion,Book
0,224517,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,161936,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,174430,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,342942,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,233078,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Clean/Combine Mechanics and Subcategories

Manually cleaning up Subcategories. This section on BGG has a lot of "catch-all" concept that involve theming, mechanics (which should be in the mechanics section), and large subcategories that should be alone. We use our domain knowledge to clean this section.

In [67]:
# print our subcategories so we can prepare to sort them
sorted(subcategories.columns)

['Abstract Strategy',
 'Action / Dexterity',
 'Adventure',
 'Age of Reason',
 'American Civil War',
 'American Indian Wars',
 'American Revolutionary War',
 'American West',
 'Ancient',
 'Animals',
 'Arabian',
 'Aviation / Flight',
 'BGGId',
 'Bluffing',
 'Book',
 'Card Game',
 "Children's Game",
 'City Building',
 'Civil War',
 'Civilization',
 'Collectible Components',
 'Comic Book / Strip',
 'Deduction',
 'Dice',
 'Economic',
 'Educational',
 'Electronic',
 'Environmental',
 'Expansion for Base-game',
 'Exploration',
 'Fan Expansion',
 'Fantasy',
 'Farming',
 'Fighting',
 'Game System',
 'Horror',
 'Humor',
 'Industry / Manufacturing',
 'Korean War',
 'Mafia',
 'Math',
 'Mature / Adult',
 'Maze',
 'Medical',
 'Medieval',
 'Memory',
 'Miniatures',
 'Modern Warfare',
 'Movies / TV / Radio theme',
 'Murder/Mystery',
 'Music',
 'Mythology',
 'Napoleonic',
 'Nautical',
 'Negotiation',
 'Novel-based',
 'Number',
 'Party Game',
 'Pike and Shot',
 'Pirates',
 'Political',
 'Post-Napoleonic'

In [69]:
# picking the items that will go under "themes"
subcats_themes = ['BGGId', 
'Adventure',
 'Age of Reason',
 'American Civil War',
 'American Indian Wars',
 'American Revolutionary War',
 'American West',
 'Ancient',
 'Animals',
 'Arabian',
 'Aviation / Flight',
 'City Building',
 'Civil War',
 'Civilization',
 'Comic Book / Strip',
 'Economic',
 'Environmental',
 'Fantasy',
 'Farming',
 'Fighting',
 'Horror',
 'Humor',
 'Industry / Manufacturing',
 'Korean War',
 'Mafia',
 'Math',
 'Mature / Adult',
 'Maze',
 'Medical',
 'Medieval',
 'Modern Warfare',
 'Movies / TV / Radio theme',
 'Murder/Mystery',
 'Music',
 'Mythology',
 'Napoleonic',
 'Nautical',
 'Novel-based',
 'Number',
 'Pike and Shot',
 'Pirates',
 'Political',
 'Post-Napoleonic',
 'Prehistoric',
 'Racing',
 'Religious',
 'Renaissance',
 'Science Fiction',
 'Space Exploration',
 'Spies/Secret Agents',
 'Sports',
 'Trains',
 'Transportation',
 'Travel',
 'Trivia',
 'Video Game Theme',
 'Vietnam War',
 'World War I',
 'World War II',
 'Zombies']

In [70]:
# picking the items that will stay as subcategories
subcats_subcategories = ['BGGId', 'Exploration', 'Miniatures', 'Territory Building', 'Card Game',
             'Educational', 'Puzzle','Collectible Components',
        'Word Game','Print & Play','Electronic']

In [71]:
# looking at our mechanics in alphabetical order so we can see where to roll in the subcategory mechanics
sorted(mechanics.columns)

['Action Points',
 'Action Queue',
 'Action Retrieval',
 'Action Timer',
 'Action/Event',
 'Advantage Token',
 'Alliances',
 'Area Majority / Influence',
 'Area Movement',
 'Area-Impulse',
 'Automatic Resource Growth',
 'BGGId',
 'Betting and Bluffing',
 'Bias',
 'Bids As Wagers',
 'Bingo',
 'Bribery',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Catch the Leader',
 'Chaining',
 'Chit-Pull System',
 'Command Cards',
 'Commodity Speculation',
 'Communication Limits',
 'Connections',
 'Constrained Bidding',
 'Contracts',
 'Cooperative Game',
 'Crayon Rail System',
 'Critical Hits and Failures',
 'Cube Tower',
 'Deck Construction',
 'Deck, Bag, and Pool Building',
 'Deduction',
 'Delayed Purchase',
 'Dexterity',
 'Dice Rolling',
 'Die Icon Resolution',
 'Different Dice Movement',
 'Drawing',
 'Elapsed Real Time Ending',
 'Enclosure',
 'End Game Bonuses',
 'Events',
 'Finale Ending',
 'Follow',
 'Force Commitment',
 'Grid Coverage',
 'Grid Movement',
 'Hand Managem

In [72]:
# drop subcategories that are none of the things
subcategories.drop(['Expansion for Base-game',  'Game System', 'Book'], axis=1, inplace=True)

In [73]:
# for mechanics that are mistakenly in the subcategories frame, make sure those mechanics are in the mechanics frame instead
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Negotiation']==1].index)
mechanics.loc[indices, 'Negotiation']=1
subcategories.drop('Negotiation', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Action / Dexterity']==1].index)
mechanics.loc[indices, 'Dexterity']=1
subcategories.drop('Action / Dexterity', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Dice']==1].index)
mechanics.loc[indices, 'Dice Rolling']=1
subcategories.drop('Dice', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Bluffing']==1].index)
mechanics.loc[indices, 'Betting and Bluffing']=1
subcategories.drop('Bluffing', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Real-time']==1].index)
mechanics.loc[indices, 'Real-Time']=1
subcategories.drop('Real-time', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Memory']==1].index)
mechanics.loc[indices, 'Memory']=1
subcategories.drop('Memory', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Deduction']==1].index)
mechanics.loc[indices, 'Deduction']=1
subcategories.drop('Deduction', axis=1, inplace=True)


In [74]:
# or for larger categories in the subcategories frame, move those to the games frame
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Wargame']==1].index)
games.loc[indices, 'Cat:War']=1
subcategories.drop('Wargame', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories["Children's Game"]==1].index)
games.loc[indices, 'Cat:Childrens']=1
subcategories.drop("Children's Game", axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Party Game']==1].index)
games.loc[indices, 'Cat:Party']=1
subcategories.drop('Party Game', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Abstract Strategy']==1].index)
games.loc[indices, 'Cat:Abstract']=1
subcategories.drop('Abstract Strategy', axis=1, inplace=True)

In [75]:
# make new data frames for THEMES and SUBCATEGORIES
themes = subcategories[subcats_themes]
themes = pd.concat([themes, themes_attach], axis=1)
subcategories = subcategories[subcats_subcategories]

Save all the file we just cleaned or created!

In [76]:
mechanics.to_pickle('data_cleaned/mechanics.pkl')

In [77]:
subcategories.to_pickle('data_cleaned/subcategories.pkl')

In [78]:
themes.to_pickle('data_cleaned/themes.pkl')

In [79]:
mechanics.to_csv('data_kaggle/mechanics.csv', index=False)
subcategories.to_csv('data_kaggle/subcategories.csv', index=False)
themes.to_csv('data_kaggle/themes.csv', index=False)

In [80]:
mechanics = pd.read_pickle('data_cleaned/mechanics.pkl')
subcategories = pd.read_pickle('data_cleaned/subcategories.pkl')
themes = pd.read_pickle('data_cleaned/themes.pkl')

In [81]:
mechanics.head()

,BGGId,Hand Management,Income,Loans,Market,Network and Route Building,Tags,Tech Trees / Tech Tracks,Variable Set-up,Action Points,Cooperative Game,Legacy Game,Point to Point Movement,Scenario / Mission / Campaign Game,Set Collection,Trading,Variable Player Powers,Action Queue,Action Retrieval,Campaign / Battle Card Driven,Card Play Conflict Resolution,Communication Limits,Critical Hits and Failures,Deck Construction,Grid Movement,Hexagon Grid,Line of Sight,Modular Board,Multi-Use Cards,Once-Per-Game Abilities,Role Playing,Simultaneous Action Selection,Solo / Solitaire Game,End Game Bonuses,Increase Value of Unchosen Resources,Tile Placement,Area-Impulse,Dice Rolling,Follow,King of the Hill,Race,Variable Phase Order,Voting,"Deck, Bag, and Pool Building",Delayed Purchase,Force Commitment,Take That,Contracts,Enclosure,Victory Points as a Resource,TableauBuilding,Area Majority / Influence,Area Movement,Events,Movement Points,Team-Based Game,Die Icon Resolution,Hidden Movement,Automatic Resource Growth,Action/Event,Advantage Token,Simulation,Sudden Death Ending,Tug of War,Ownership,Track Movement,Grid Coverage,Pattern Building,Zone of Control,Alliances,Passed Action Token,Layering,Hidden Roles,Interrupts,Player Elimination,Semi-Cooperative Game,Traitor Game,Map Addition,Narrative Choice / Paragraph,Pick-up and Deliver,Push Your Luck,Hidden Victory Points,Investment,Chaining,Map Deformation,Stat Check Resolution,Resource to Move,Square Grid,Trick-taking,Negotiation,Score-and-Reset Game,Catch the Leader,Programmed Movement,Map Reduction,Memory,Moving Multiple Units,Storytelling,Stock Holding,Bias,Commodity Speculation,Slide/Push,Secret Unit Deployment,Constrained Bidding,Mancala,Chit-Pull System,Pattern Recognition,Highest-Lowest Scoring,Rondel,Neighbor Scope,Deduction,Paper-and-Pencil,Selection Order Bid,Targeted Clues,Connections,Ladder Climbing,Betting and Bluffing,Order Counters,Bingo,Line Drawing,Three Dimensional Movement,Movement Template,Re-rolling and Locking,Roll / Spin and Move,Resource Queue,Minimap Resolution,Multiple Maps,Command Cards,Bribery,Predictive Bid,Prisoner's Dilemma,"I Cut, You Choose",Finale Ending,Roles with Asymmetric Information,Random Production,Real-Time,Kill Steal,Relative Movement,Cube Tower,Move Through Deck,Melding and Splaying,Pattern Movement,Pieces as Map,Drawing,Static Capture,Elapsed Real Time Ending,Ordering,Lose a Turn,Bids As Wagers,Ratio / Combat Results Table,Spelling,Different Dice Movement,Physical Removal,Measurement Movement,Single Loser Game,Player Judge,Action Timer,Induction,Speed Matching,Matching,Questions and Answers,Crayon Rail System,Impulse Movement,Dexterity,Physical
0,224517,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,161936,1,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,174430,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,342942,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [82]:
subcategories.head()

,BGGId,Exploration,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic
0,224517,0,0,0,0,0,0,0,0,0,0
1,161936,0,0,0,0,0,0,0,0,0,0
2,174430,1,1,0,0,0,0,0,0,0,0
3,342942,0,0,0,0,0,0,0,0,0,0
4,233078,1,0,0,0,0,0,0,0,0,0


In [83]:
themes.head()

,BGGId,Adventure,Age of Reason,American Civil War,American Indian Wars,American Revolutionary War,American West,Ancient,Animals,Arabian,Aviation / Flight,City Building,Civil War,Civilization,Comic Book / Strip,Economic,Environmental,Fantasy,Farming,Fighting,Horror,Humor,Industry / Manufacturing,Korean War,Mafia,Math,Mature / Adult,Maze,Medical,Medieval,Modern Warfare,Movies / TV / Radio theme,Murder/Mystery,Music,Mythology,Napoleonic,Nautical,Novel-based,Number,Pike and Shot,Pirates,Political,Post-Napoleonic,Prehistoric,Racing,Religious,Renaissance,Science Fiction,Space Exploration,Spies/Secret Agents,Sports,Trains,Transportation,Travel,Trivia,Video Game Theme,Vietnam War,World War I,World War II,Zombies,Theme_Food / Cooking,Theme_Anime / Manga,Theme_Superheroes,Theme_Anthropomorphic Animals,Theme_Mystery / Cri,Theme_Alternate History,Theme_Cthulhu Mythos,Theme_Vikings,Theme_Colonial,Theme_Post-Apocalyptic,Theme_Robots,Theme_Time Travel,Theme_Witches,Theme_Archaeology / Paleontology,Theme_Sieg,Theme_Mad Science / Mad Scientist,Theme_Art,Theme_Natur,Theme_Pirates,Theme_Mining,Theme_Gardening,Theme_Retro,Theme_Steampunk,Theme_Burglary and Heists,Theme_Native Americans / First Peoples,Theme_Deserts,Theme_Flowers,Theme_Fishing / Angling,Theme_Cyberpunk,Theme_Submarines,Theme_Construction,Theme_Ninjas,Theme_Circus,Theme_Amusement Parks / Theme Parks,Theme_Tropical,Theme_King Arthur / The Knights of the Round Table / Camelot,Theme_Spooky Old Houses,Theme_Biology,Theme_Love / Romanc,Theme_Gladiators,Theme_Kaiju,Theme_Samurai,Theme_Weather,Theme_Villainy,Theme_Books / Libraries,Theme_Fantasy Sports,Theme_Safaris,Theme_Boss Battl,Theme_City,Theme_Arcade Video Games,Theme_Airships / Blimps / Dirigibles / Zeppelins,Theme_Mech Warfar,Theme_Alchemy,Theme_Survival,Theme_Movie Industry,Theme_Oil / Gas / Petroleu,Theme_Scienc,Theme_Dreams / Nightmares,Theme_Boardgaming,Theme_Polic,Theme_Excrements / Feces / Poop / Urin,Theme_Art style – Art Deco,Theme_Beaches,Theme_Trees and Forests,Theme_Psychology,Theme_Climate Chang,Theme_Animal Battles,Theme_Music Making & Makers,Theme_Nuclear option,Theme_Ecology,Theme_Tropical Islands,Theme_Evolution,Theme_Under the Sea,Theme_Video Game Theme: Nintendo,Theme_Mail / Stamps / The Post Offic,Theme_Firefighting,Theme_Romance of the Three Kingdoms,Theme_Volcanoes,Theme_Fictional Games,Theme_Hell,Theme_Attorneys / Courts,Theme_Jail / Prison (Modern),Theme_Treasure Diving,Theme_Hackers,Theme_Sci-Fi Sports,Theme_Aztecs,Theme_Astronomy,Theme_Helicopters,Theme_Chibis,Theme_Mayans,Theme_Trucks,Theme_Chemistry,Theme_Computer / Information Technology Industry,Theme_Automotive Industry,Theme_Sewing / Knitting / Cloth-Making,Theme_US National Parks,Theme_Trash / Garbag,Theme_Movies,Theme_Dolls,Theme_Mushrooms,Theme_Motorcycles,Theme_Hot Air Balloons,Theme_Plague Epidemic,Theme_Art style – Art Nouveau,"Theme_Pub, Bars, Bistros...",Theme_Laundry & Socks,Theme_Smuggling,Theme_Chivalry / Jousting / Tournaments (Medieval Europe),Theme_Canals,Theme_Silk Road,Theme_TV Detectives,Theme_Stock Market,"Theme_Zoos, Aquaria, Safari Parks",Theme_Environmental Protection / Degradation / Pollution,Theme_Journalis,Theme_My Best Lif,Theme_Battle Royal,Theme_Camping,Theme_Memes,Theme_African Americans,Theme_Traffic / Driving,Theme_Physics,Theme_Jewelry,Theme_Psychic Powers,Theme_Druids,Theme_UFOs,Theme_Cannibals / Cannibalis,Theme_Hanseatic Leagu,Theme_Cacti and Succulents,Theme_Floating islands in the sky,Theme_Cereal Games,Theme_Cemeteries / Graveyards,Theme_Endangered / Extinct species,Theme_Photography,Theme_Business/Management,Theme_Hiking,Theme_Infiltration,Theme_Video Game Theme: Tetris,Theme_Video Game Theme: Pokémon,Theme_Pulp,Theme_Video Game Theme: Super Mario Bros.,Theme_FIFA World Cup,Theme_Teaching Programming,Theme_Fine Art and Art Museums,Theme_Television (TV) Industry,Theme_Cryptids,Theme_Disney Theme Parks,Theme_Templ,Theme_Perfu,Theme_Māori,Theme_Fireworks,Theme_Rubik's Cub,Theme_Astrology,Theme_Fashion,Theme_School /

## Designers

In [84]:
# Load up our designers file!
designers = pd.read_pickle('data_dirty/designers.pkl')
designers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 16542 entries, BGGId to Tommy Paupe
dtypes: float64(16541), int64(1)
memory usage: 5.0 GB


In [85]:
designers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
designers.fillna(0, inplace=True) # Fill NaN

# change all flags to int8
temp_id = designers['BGGId'] 
designers.drop('BGGId', axis=1, inplace=True)
designers = designers.astype('int8')
designers['BGGId'] = temp_id

designers.reset_index(inplace=True, drop=True)# reset index
designers.info() # check memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 16542 entries, Gavan Brown to BGGId
dtypes: int64(1), int8(16541)
memory usage: 643.2 MB


In [86]:
# save file of all possible designers
designers.to_pickle('data_cleaned/designers_all.pkl')
designers.to_csv('data_kaggle/designers_all.csv', index=False)

In [87]:
# locate all row that sum to 3 or less (find low experience designers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = designers.loc[:, designers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience designers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp designer
designers['Low-Exp Designer'] = 0

# for each index in the low exp list, set low exp designer to 1
for index in indices:
    designers.loc[index, 'Low-Exp Designer'] = 1
    
# drop all columns for one-off designers
designers.drop(designers.loc[:, designers.sum(axis=0) <= 3], axis=1, inplace=True)

In [88]:
# save file of designers reduced to 3 or more works
designers.to_pickle('data_cleaned/designers_reduced.pkl')
designers.to_csv('data_kaggle/designers_reduced.csv', index=False)

In [89]:
designers = pd.read_pickle('data_cleaned/designers_reduced.pkl')
designers.head()

Gavan Brown  Matt Tolman  Martin Wallace  Rob Daviau  Matt Leacock  \
0            1            1               1           0             0   
1            0            0               0           1             1   
2            0            0               0           0             0   
3            0            0               0           0             0   
4            0            0               0           0             0   

   Isaac Childres  Corey Konieczka  Christian T. Petersen  Paul Dennen  \
0               0                0                      0            0   
1               0                0                      0            0   
2               1                0                      0            0   
3               0                0                      0            0   
4               0                1                      1            0   

   Jacob Fryxelius  Roberto Di Meglio  Marco Maggi  Francesco Nepitello  \
0                0                  0            0                    0   
1                0                  0            0                    0   
2                0                  0            0                    0   
3                0                  0            0                    0   
4                0                  0            0                    0   

   R. Eric Reuss  Jens Drögemüller  Helge Ostertag  Jason Matthews  \
0              0                 0               0               0   
1              0                 0               0               0   
2              0                 0               0               0   
3              0                 0               0               0   
4              0                 0               0               0   

   Vlaada Chvátil  Alexander Pfister  Stefan Feld  Jamey Stegmaier  \
0               0                  0            0                0   
1               0                  0            0                0   
2               0                  0            0                0   
3               0                  0            0                0   
4               0                  0            0                0   

   Touko Tahkokallio  Antoine Bauza  Bruno Cathala  Adam Kwapiński  \
0                  0              0              0               0   
1                  0              0              0               0   
2                  0              0              0               0   
3                  0              0              0               0   
4                  0              0              0               0   

   Mac Gerdts  Uwe Rosenberg  Elizabeth Hargrave  Nate French  Cole Wehrle  \
0           0              0                   0            0            0   
1           0              0                   0            0            0   
2           0              0                   0            0            0   
3           0              0                   0            0            0   
4           0              0                   0            0            0   

   Reiner Stockhausen  Josh J. Carlson  Adam Carlson  James A. Wilson  \
0                   0                0             0                0   
1                   0                0             0                0   
2                   0                0             0                0   
3                   0                0             0                0   
4                   0                0             0                0   

   Simone Luciani  Thomas Sing  Alan Stone  Jeroen Doumen  Joris Wiersinga  \
0               0            0           0              0                0   
1               0            0           0              0                0   
2               0            0           0              0                0   
3               0            0           0              0                0   
4               0            0           0              0                0   

   Vladimír Suchý  Andreas S

## Artists

In [90]:
# load artists file
artists = pd.read_pickle('data_dirty/artists.pkl')
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 15835 entries, BGGId to Clara Louise Martin
dtypes: float64(15834), int64(1)
memory usage: 4.8 GB


In [91]:
artists.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
artists.fillna(0, inplace=True) # fill nan

# convert flags to int8
temp_id = artists['BGGId']
artists.drop('BGGId', axis=1, inplace=True)
artists = artists.astype('int8')
artists['BGGId'] = temp_id

artists.reset_index(inplace=True, drop=True)# reset index
artists.info()# get memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 15835 entries, Gavan Brown to BGGId
dtypes: int64(1), int8(15834)
memory usage: 615.7 MB


In [92]:
# save all artists to file
artists.to_pickle('data_cleaned/artists_all.pkl')
artists.to_csv('data_kaggle/artists_all.csv', index=False)

In [93]:
# locate all row that sum to 3 or less (find low experience artists)

# change the 3 to whatever desired for more or less experience
lowexp_rows = artists.loc[:, artists.sum(axis=0) <= 3]

# Locate the columns that contain the low experience artists
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Artist
artists['Low-Exp Artist'] = 0

# for each index in the low exp list, set low exp Artist to 1
for index in indices:
    artists.loc[index, 'Low-Exp Artist'] = 1
    
# drop all columns for one-off artists
artists.drop(artists.loc[:, artists.sum(axis=0) <= 3], axis=1, inplace=True)

In [94]:
# save artists to file reduces to 3 or more works
artists.to_pickle('data_cleaned/artists_reduced.pkl')
artists.to_csv('data_kaggle/artists_reduced.csv', index=False)

In [95]:
artists = pd.read_pickle('data_cleaned/artists_reduced.pkl')
artists.head()

Gavan Brown  Lina Cossette  David Forest  Damien Mammoliti  \
0            1              1             1                 1   
1            0              0             0                 0   
2            0              0             0                 0   
3            0              0             0                 0   
4            0              0             0                 0   

   Chris Quilliams  Alexandr Elichev  Josh T. McDowell  Loïc Billiau  \
0                0                 0                 0             0   
1                1                 0                 0             0   
2                0                 1                 1             0   
3                0                 0                 0             1   
4                0                 0                 0             0   

   Dennis Lohausen  Christof Tisch  Scott Schomburg  Clay Brooks  Raul Ramos  \
0                0               0                0            0           0   
1                0               0                0            0           0   
2                0               0                0            0           0   
3                1               1                0            0           0   
4                0               0                1            0           0   

   Nate Storm  Isaac Fryxelius  Daniel Fryxelius  John Howe  Fabio Maiorana  \
0           0                0                 0          0               0   
1           0                0                 0          0               0   
2           0                0                 0          0               0   
3           0                0                 0          0               0   
4           0                0                 0          0               0   

   Francesca Baerald  David Demaret  David Ardila  Tiziano Baracchi  \
0                  0              0             0                 0   
1                  0              0             0                 0   
2                  0              0             0                 0   
3                  0              0             0                 0   
4                  0              0             0                 0   

   Ryan Barger  Jon Bosco  Matt Bradbury  Alexandre Dainche  Tony Foti  \
0            0          0              0                  0          0   
1            0          0              0                  0          0   
2            0          0              0                  0          0   
3            0          0              0                  0          0   
4            0          0              0                  0          0   

   Mariusz Gandzel  Zach Graves  Jeff Lee Johnson  Jason Juta  \
0                0            0                 0           0   
1                0            0                 0           0   
2                0            0                 0           0   
3                0            0                 0           0   
4                0            0                 0           0   

   Ignacio Bazán Lazcano  Mark Molnar  Jake Murray  Ameen Naksewee  \
0                      0            0            0               0   
1                      0            0            0               0   
2                      0            0            0               0   
3                      0            0            0               0   
4                      0            0            0               0   

   David Auden Nash  Mike Nash  Maciej Rebisz  Alexandru Sabo  Stephen Somers  \
0                 0          0              0               0               0   
1                 0          0              0               0               0   
2                 0          0              0               0               0   
3                 0          0              0               0               0   
4                 0          0              0               0               0   

   Matthew Starbuck  Nicholas Stohlman  Angela Sung 

## Publishers

In [96]:
# load publishers
publishers = pd.read_pickle('data_dirty/publishers.pkl')
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 10994 entries, BGGId to Play Bishop LLC
dtypes: float64(10993), int64(1)
memory usage: 3.3 GB


In [97]:
publishers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
publishers.fillna(0, inplace=True)# fill nan

# set flags to int8
temp_id = publishers['BGGId']
publishers.drop('BGGId', axis=1, inplace=True)
publishers = publishers.astype('int8')
publishers['BGGId'] = temp_id

publishers.reset_index(inplace=True, drop=True) # reset index
publishers.info() # get memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40756 entries, 0 to 40755
Columns: 10994 entries, Roxley to BGGId
dtypes: int64(1), int8(10993)
memory usage: 427.6 MB


In [98]:
# save all publishers to file
publishers.to_pickle('data_cleaned/publishers_all.pkl')
publishers.to_csv('data_kaggle/publishers_all.csv', index=False)

In [99]:
# locate all row that sum to 3 or less (find low experience publishers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = publishers.loc[:, publishers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience publishers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Publisher
publishers['Low-Exp Publisher'] = 0

# for each index in the low exp list, set low exp Publisher to 1
for index in indices:
    publishers.loc[index, 'Low-Exp Publisher'] = 1
    
# drop all columns for one-off publishers
publishers.drop(publishers.loc[:, publishers.sum(axis=0) <= 3], axis=1, inplace=True)

In [100]:
# save publishers reduced to 3 or more works
publishers.to_pickle('data_cleaned/publishers_reduced.pkl')
publishers.to_csv('data_kaggle/publishers_reduced.csv', index=False)

In [101]:
publishers = pd.read_pickle('data_cleaned/publishers_reduced.pkl')
publishers.head()

Roxley  Arclight Games  Board Game Rookie  BoardM Factory  \
0       1               1                  1               1   
1       0               0                  0               0   
2       0               1                  0               0   
3       0               0                  0               0   
4       0               1                  0               0   

   CMON Global Limited  Conclave Editora  CrowD Games  Dexker Games  Funforge  \
0                    1                 1            1             1         1   
1                    0                 0            0             0         0   
2                    0                 0            0             0         0   
3                    1                 0            0             0         0   
4                    0                 0            0             0         0   

   Gém Klub Kft.  Ghenos Games  Giant Roc  Lanlalen  Lord of Boards  \
0              1             1          1         1               1   
1              1             0          0         0               0   
2              1             0          0         0               0   
3              1             0          0         0               0   
4              0             0          0         0               0   

   Maldito Games  PHALANX  Rebel Sp. z o.o.  TLAMA games  White Goblin Games  \
0              1        1                 1            1                   1   
1              0        0                 0            0                   0   
2              0        0                 0            0                   0   
3              1        0                 0            0                   1   
4              0        0                 0            0                   0   

   盒拍工作室 Hepa Studio  Z-Man Games  Asterion Press  Devir  Filosofia Éditions  \
0                  1            0               0      0                   0   
1                  0            1               1      1                   1   
2                  0            0               0      0                   0   
3                  0            0               0      0                   0   
4                  0            0               1      0                   0   

   Hobby Japan  Jolly Thinkers  Korea Boardgames  Lacerta  \
0            0               0                 0        0   
1            1               1                 1        1   
2            0               0                 1        0   
3            0               0                 1        0   
4            0               0                 0        0   

   Lifestyle Boardgames Ltd  MINDOK  Ігромаг  Cephalofair Games  Albi  \
0                         0       0        0                  0     0   
1                         1       1        1                  0     0   
2                         0       0        0                  1     1   
3                         0       1        0                  0     0   
4                         0       0        0                  0     0   

   Albi Polska  Feuerland Spiele  Galápagos Jogos  Games Warehouse  \
0            0                 0                0                0   
1            0                 0                0                0   
2            1                 1                1                1   
3            0                 1                0                0   
4            0                 0                1                0   

   Hobby World  MYBG Co., Ltd.  Capstone Games  Cranio Creations  Game Harbor  \
0            0               0               0                 0            0   
1            0               0               0                 0            0   
2            1               1               0                 0            0   
3            0               0               1                 1            1   
4            1               0               0                 0            0   

   Grok Games  IGAMES  Lauta

# Ratings - by Item, User, & Comments

## Test Code

In [ ]:
master_comments = pd.read_pickle('data_dirty/raw_game_ratings.pkl')

# integer_reduce
master_comments['BGGId'] = master_comments['BGGId'].astype(int)
master_comments['Rating'] = master_comments['Rating'].astype(float)

master_comments.head()

In [ ]:
master_comments.drop_duplicates(keep='first', inplace=True)

In [ ]:
master_comments.info()

In [ ]:
nodrops = list(master_comments.loc[master_comments.Comments!=''].index)

In [ ]:
comments_only = master_comments.iloc[nodrops]

In [ ]:
comments_only.head(10)

In [ ]:
comments_only.to_pickle('data_cleaned/master_comments_file.pkl')

In [ ]:
del comments_only
gc.collect()

## Deploy - User and Item Ratings

In [ ]:
raw_game_ratings = pd.read_pickle('data_dirty/raw_game_ratings.pkl')

In [ ]:
ratings = raw_game_ratings[['BGGId', 'Rating', 'Username']]
ratings.head()

In [ ]:
del raw_game_ratings
gc.collect()

In [ ]:
ratings.drop_duplicates(keep='first', inplace=True)

In [ ]:
ratings.head()

In [ ]:
ratings.to_csv('data_kaggle/user_ratings.csv', index=False)

In [ ]:
ratings.Username.nunique()

In [ ]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current.pkl')
game_ids = list(game_ids_current)

In [ ]:
user_ratings = {}

raw_ratings = {}

for item in game_ids:
    raw_ratings[item] = []

In [ ]:
ratings.apply(lambda x: process_dataframe_ratings(x, user_ratings, raw_ratings), axis=1)

In [ ]:
user_ratings['Threnody']

In [ ]:
raw_ratings[213788]

In [ ]:
# save dictionary
with open('data_cleaned/game_raw_ratings.json', 'w') as convert_file:
    convert_file.write(json.dumps(raw_ratings))
    
with open('real_ratings/user_ratings_unscaled.json', 'w') as convert_file:
    convert_file.write(json.dumps(user_ratings))
    
del ratings
gc.collect() 

### Winnow Users to 5+ Ratings

In [ ]:
# Opening JSON file
with open('data_cleaned/user_ratings_unscaled.json') as json_file:
    user_ratings = json.load(json_file)

In [ ]:
all_users = list(user_ratings.keys())

In [ ]:
for user in all_users:

    if len(user_ratings[user]) < 5:
        print("Removing user "+user)
        del user_ratings[user]
        continue

In [ ]:
with open('data_cleaned/user_ratings_unscaled.json', 'w') as convert_file:
    convert_file.write(json.dumps(user_ratings))

## Ratings Distribution

In [ ]:
# Load the storage dictionary for this block
with open('data_cleaned/game_raw_ratings.json') as json_file:
    raw_ratings = json.load(json_file) 

In [ ]:
len(raw_ratings)

In [ ]:
ratings_distribution = pd.DataFrame()

for item in raw_ratings.keys():
    
    print(item)
    
    ratings_temp = pd.DataFrame(raw_ratings[item]).round(1)
    ratings_counts = pd.DataFrame(ratings_temp.value_counts()).sort_index().T
    
    ratings_distribution = ratings_distribution.append(ratings_counts)

In [ ]:
#ratings_distribution = pd.read_pickle('data_cleaned/ratings_distribution.pkl')

In [ ]:
ratings_distribution.set_axis(list(raw_ratings.keys()), axis=0, inplace=True)

In [ ]:
ratings_distribution.head()

In [ ]:
ratings_distribution.fillna(0, inplace=True)

In [ ]:
ratings_distribution.head()

In [ ]:
ratings_distribution['total_ratings'] = ratings_distribution.sum(axis=1)

In [ ]:
#ratings_distribution = ratings_distribution.T.reset_index().T

In [ ]:
ratings_distribution.reset_index(inplace=True)

In [ ]:
ratings_distribution.head()

In [ ]:
ratings_distribution.rename(columns={'index':'BGGId'}, inplace=True)

In [ ]:
ratings_distribution['BGGId'] = ratings_distribution['BGGId'].astype('int64')

In [ ]:
ratings_distribution.head()

In [ ]:
ratings_distribution.to_pickle('data_cleaned/ratings_distribution.pkl')

In [ ]:
ratings_distribution.to_csv('data_kaggle/ratings_distribution.csv', index=False)

## Item Means



In [ ]:
ratings = pd.read_pickle('real_ratings/real_user_ratings_unscaled_fullmatrix.pkl')

In [ ]:
ratings.head()

In [ ]:
ratings = ratings.replace(0, np.NaN)

In [ ]:
item_means = ratings.mean().to_dict()

In [ ]:
# save catalog to file
with open('data_cleaned/item_means.json', 'w') as convert_file:
     convert_file.write(json.dumps(item_means))

## User Means

In [ ]:
# Opening JSON file
with open('real_ratings/real_user_ratings_unscaled.json') as json_file:
    user_ratings = json.load(json_file)

In [ ]:
len(user_ratings)

In [ ]:
user_means = {}

In [ ]:
for person in user_ratings:
    user_items = []
    for item in user_ratings[person]:
        user_items.append(user_ratings[person][item])
    user_mean = round((mean(user_items)), 1)
    user_means[person] = user_mean

In [ ]:
user_means['Threnody']

In [ ]:
user_means['moosh21']

In [ ]:
user_means['Shade92008']

In [ ]:
user_means['Torsten']

In [ ]:
# save dictionary
with open('data_cleaned/user_means.json', 'w') as convert_file:
    convert_file.write(json.dumps(user_means))

In [ ]:
del user_means
gc.collect()

In [ ]:
# Opening JSON file
with open('data_cleaned/user_means.json') as json_file:
    user_means_dict = json.load(json_file)

In [ ]:
user_means = pd.DataFrame.from_dict(user_means_dict, orient='index')
user_means.rename(columns={0:'Mean'}, inplace=True)
user_means.head()

In [ ]:
user_means.to_pickle('data_cleaned/user_means.pkl')